<a href="https://colab.research.google.com/github/Krisanbee/Sentiment-Analysis---Major-Project/blob/main/Sentiment_Analysis_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-plot

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk import tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import (accuracy_score,confusion_matrix)

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import scikitplot as skplt

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [ ]:
!unzip "/content/drive/My Drive/ML Practise/Major Project/1429_1.csv (1).zip"

Archive:  /content/drive/My Drive/ML Practise/Major Project/1429_1.csv (1).zip
replace 1429_1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
header_names = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','rating','16','review','18','19','20','21']
df = pd.read_csv('/content/drive/My Drive/ML Practise/Major Project/1429_1.csv (1).zip',header=None,skiprows=1,names=header_names)
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,rating,16,review,18,19,20,21
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [ ]:
df1=df.drop(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','16','18','19','20','21'],axis=1)
df1.head()

,rating,review
0,5.0,This product so far has not disappointed. My c...
1,5.0,great for beginner or experienced person. Boug...
2,5.0,Inexpensive tablet for him to use and learn on...
3,4.0,I've had my Fire HD 8 two weeks now and I love...
4,5.0,I bought this for my grand daughter when she c...


In [ ]:
df1['label'] = df1['rating'].apply(lambda x: 'pos' if x>3 else 'neu' if x==3 else 'neg')
df1.head()

,rating,review,label
0,5.0,This product so far has not disappointed. My c...,pos
1,5.0,great for beginner or experienced person. Boug...,pos
2,5.0,Inexpensive tablet for him to use and learn on...,pos
3,4.0,I've had my Fire HD 8 two weeks now and I love...,pos
4,5.0,I bought this for my grand daughter when she c...,pos


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rating  34627 non-null  float64
 1   review  34659 non-null  object 
 2   label   34660 non-null  object 
dtypes: float64(1), object(2)
memory usage: 812.5+ KB


In [ ]:
df1.review = df1.review.astype(str)

In [ ]:
df1.review = df1.review.apply(lambda x:x.lower())
df1.head()

,rating,review,label
0,5.0,this product so far has not disappointed. my c...,pos
1,5.0,great for beginner or experienced person. boug...,pos
2,5.0,inexpensive tablet for him to use and learn on...,pos
3,4.0,i've had my fire hd 8 two weeks now and i love...,pos
4,5.0,i bought this for my grand daughter when she c...,pos


In [ ]:
def remove_punctuations(text):
  for punctuation in string.punctuation:
    text = text.replace(punctuation,'')
  return text  

In [ ]:
df1.review = df1.review.apply(remove_punctuations)
df1.head()

,rating,review,label
0,5.0,this product so far has not disappointed my ch...,pos
1,5.0,great for beginner or experienced person bough...,pos
2,5.0,inexpensive tablet for him to use and learn on...,pos
3,4.0,ive had my fire hd 8 two weeks now and i love ...,pos
4,5.0,i bought this for my grand daughter when she c...,pos


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop = stopwords.words('english')

In [ ]:
df1.review = df1.review.apply(lambda x: " ".join(x for x in x.split() if x not in stop ))

In [ ]:
df1.head()

,rating,review,label
0,5.0,product far disappointed children love use lik...,pos
1,5.0,great beginner experienced person bought gift ...,pos
2,5.0,inexpensive tablet use learn step nabi thrille...,pos
3,4.0,ive fire hd 8 two weeks love tablet great valu...,pos
4,5.0,bought grand daughter comes visit set user ent...,pos


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df1['review_tokenized'] = df1.review.apply(lambda x: word_tokenize(x))

In [ ]:
df1['review_tokenized'].head()

0    [product, far, disappointed, children, love, u...
1    [great, beginner, experienced, person, bought,...
2    [inexpensive, tablet, use, learn, step, nabi, ...
3    [ive, fire, hd, 8, two, weeks, love, tablet, g...
4    [bought, grand, daughter, comes, visit, set, u...
Name: review_tokenized, dtype: object

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
df1['review_lemmatized'] = df1.review_tokenized.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [ ]:
df1.review_lemmatized.head()

0    [product, far, disappointed, child, love, use,...
1    [great, beginner, experienced, person, bought,...
2    [inexpensive, tablet, use, learn, step, nabi, ...
3    [ive, fire, hd, 8, two, week, love, tablet, gr...
4    [bought, grand, daughter, come, visit, set, us...
Name: review_lemmatized, dtype: object

In [ ]:
df1.head()

,rating,review,label,review_tokenized,review_lemmatized
0,5.0,product far disappointed children love use lik...,pos,"[product, far, disappointed, children, love, u...","[product, far, disappointed, child, love, use,..."
1,5.0,great beginner experienced person bought gift ...,pos,"[great, beginner, experienced, person, bought,...","[great, beginner, experienced, person, bought,..."
2,5.0,inexpensive tablet use learn step nabi thrille...,pos,"[inexpensive, tablet, use, learn, step, nabi, ...","[inexpensive, tablet, use, learn, step, nabi, ..."
3,4.0,ive fire hd 8 two weeks love tablet great valu...,pos,"[ive, fire, hd, 8, two, weeks, love, tablet, g...","[ive, fire, hd, 8, two, week, love, tablet, gr..."
4,5.0,bought grand daughter comes visit set user ent...,pos,"[bought, grand, daughter, comes, visit, set, u...","[bought, grand, daughter, come, visit, set, us..."


In [ ]:
df1.review_lemmatized = df1.review_lemmatized.astype(str)

In [ ]:
x = df1.iloc[:,4].values
y = df1.iloc[:,2].values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(25995,)
(8665,)
(25995,)
(8665,)


In [ ]:
final = Pipeline([('vect',TfidfVectorizer()),
                            ('model',SVC())])

In [ ]:
np.unique(y_train,return_counts=True)

(array(['neg', 'neu', 'pos'], dtype=object), array([  648,  1141, 24206]))

In [ ]:
final.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', de

In [ ]:
y_pred = final.predict(x_test)

In [ ]:
accuracy_score(y_pred,y_test)

0.936064627813041

In [ ]:
confusion_matrix(y_pred,y_test)

array([[   2,    0,    1],
       [   0,    0,    0],
       [ 195,  358, 8109]])